# Тестовоe задание на позицию junior-аналитика в Банк ВТБ

Из видео https://www.youtube.com/watch?v=vG6_1zIiuao от Simulative.
Файлы: https://drive.google.com/drive/folders/13I9_shAO3Qtj04nKDB4q7FazAXKyMu_x

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql

In [2]:
with sql.connect('../data_ignored/database.db') as con:
    pl_df = pd.read_sql_query('SELECT * FROM pl', con=con)

pl_df.head()

,client_id,segment,period,product,pl
0,ZDosW1zUAi,Массовый,p_1,Кредит,9.018067
1,wVRHQ9hzrw,Массовый,p_1,Кредит,18.889585
2,6DZQIaOyL3,Массовый,p_1,Кредит,0.369257
3,jDwO0sMhL2,Массовый,p_1,Кредит,124.315319
4,FXIJUkgjLy,Массовый,p_1,Кредит,41.287831


In [3]:
with sql.connect('../data_ignored/database.db') as con:
    campaign = pd.read_sql_query('SELECT * FROM campaign', con=con)

campaign.head()

,client_id,product,response,manager_id
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA
1,HkhuPN1KG7,Кредит,Не обработано менеджером,CM_PzEEav4ic0
2,x1rEdC5fys,Кредит,Согласие,CM_RCwv1r0OHO
3,fx0acVADvj,Кредит,Согласие,CM_nJ7j5pAcMB
4,nDNNydkKQR,Кредит,Согласие,CM_EiBeeRMhdA


In [6]:
pl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92609 entries, 0 to 92608
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   client_id  92609 non-null  object 
 1   segment    92609 non-null  object 
 2   period     92609 non-null  object 
 3   product    92609 non-null  object 
 4   pl         92609 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.5+ MB


In [7]:
len(pl_df.groupby(['client_id', 'period', 'product']))

92609

Один и тот же клиент не может быть в 2 разных сегментах, но может быть в разных продуктах и периодах

In [8]:
pl_df.nunique()

client_id    24992
segment          3
period           2
product          5
pl           92609
dtype: int64

In [9]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   client_id   5000 non-null   object
 1   product     5000 non-null   object
 2   response    5000 non-null   object
 3   manager_id  5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [10]:
campaign.nunique()

client_id     5000
product          1
response         5
manager_id      20
dtype: int64

In [25]:
campaign['product'].unique()

array(['Кредит'], dtype=object)

Рекламная кампания проводилась только по продукту "Кредит"

#### 1
Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.

In [4]:
pl_df.groupby(['segment', 'product', 'period']).agg({'pl': 'mean', 'client_id': 'count'})

pl  client_id
segment     product          period                       
Верхний     Банковская карта p_1      60.571545         49
                             p_2      51.770009        167
            Депозит          p_1     112.474520        254
                             p_2      99.260623        283
            Кредит           p_1     236.938710        360
                             p_2     248.191513        548
            РКО              p_1      22.847632       1149
                             p_2      22.064079       1162
            Эквайринг        p_1      80.517184        183
                             p_2      72.968866        284
Массовый    Банковская карта p_1      13.310203        895
                             p_2      11.726557       2636
            Депозит          p_1      28.747545       3537
                             p_2      25.219855       4345
            Кредит           p_1      59.219407       5247
                             p_2      67.109774       7845
            РКО              p_1       5.795114      16728
                             p_2       5.566726      16700
            Эквайринг        p_1      19.627608       2625
                             p_2      17.348344       4424
Стандартный Банковская карта p_1      26.421616        338
                             p_2      22.436106        917
            Депозит          p_1      60.994262       1292
                             p_2      49.332790       1570
            Кредит           p_1     120.101219       1871
                             p_2     131.611788       2871
            РКО              p_1      11.418812       5907
                             p_2      10.983568       5870
            Эквайринг        p_1      39.329551        962
                             p_2      35.046014       1590

In [5]:
pl_df.pivot_table(['pl', 'client_id'], ['segment', 'product'], 'period', aggfunc=['mean', 'count'])

mean                 count                \
                                      pl             client_id            pl   
period                               p_1         p_2       p_1    p_2    p_1   
segment     product                                                            
Верхний     Банковская карта   60.571545   51.770009        49    167     49   
            Депозит           112.474520   99.260623       254    283    254   
            Кредит            236.938710  248.191513       360    548    360   
            РКО                22.847632   22.064079      1149   1162   1149   
            Эквайринг          80.517184   72.968866       183    284    183   
Массовый    Банковская карта   13.310203   11.726557       895   2636    895   
            Депозит            28.747545   25.219855      3537   4345   3537   
            Кредит             59.219407   67.109774      5247   7845   5247   
            РКО                 5.795114    5.566726     16728  16700  16728   
            Эквайринг          19.627608   17.348344      2625   4424   2625   
Стандартный Банковская карта   26.421616   22.436106       338    917    338   
            Депозит            60.994262   49.332790      1292   1570   1292   
            Кредит            120.101219  131.611788      1871   2871   1871   
            РКО                11.418812   10.983568      5907   5870   5907   
            Эквайринг          39.329551   35.046014       962   1590    962   

                                     
                                     
period                          p_2  
segment     product                  
Верхний     Банковская карта    167  
            Депозит             283  
            Кредит              548  
            РКО                1162  
            Эквайринг           284  
Массовый    Банковская карта   2636  
            Депозит            4345  
            Кредит             7845  
            РКО               16700  
            Эквайринг          4424  
Стандартный Банковская карта    917  
            Депозит            1570  
            Кредит             2871  
            РКО                5870  
            Эквайринг          1590

In [6]:
pl_df.pivot_table(['pl', 'client_id'], ['segment', 'product'], 'period', aggfunc={'pl': ['mean', 'sum'], 'client_id': 'count'})

client_id                 pl              \
                                 count               mean               
period                             p_1    p_2         p_1         p_2   
segment     product                                                     
Верхний     Банковская карта        49    167   60.571545   51.770009   
            Депозит                254    283  112.474520   99.260623   
            Кредит                 360    548  236.938710  248.191513   
            РКО                   1149   1162   22.847632   22.064079   
            Эквайринг              183    284   80.517184   72.968866   
Массовый    Банковская карта       895   2636   13.310203   11.726557   
            Депозит               3537   4345   28.747545   25.219855   
            Кредит                5247   7845   59.219407   67.109774   
            РКО                  16728  16700    5.795114    5.566726   
            Эквайринг             2625   4424   19.627608   17.348344   
Стандартный Банковская карта       338    917   26.421616   22.436106   
            Депозит               1292   1570   60.994262   49.332790   
            Кредит                1871   2871  120.101219  131.611788   
            РКО                   5907   5870   11.418812   10.983568   
            Эквайринг              962   1590   39.329551   35.046014   

                                                            
                                        sum                 
period                                  p_1            p_2  
segment     product                                         
Верхний     Банковская карта    2968.005722    8645.591583  
            Депозит            28568.528169   28090.756271  
            Кредит             85297.935527  136008.949279  
            РКО                26251.929445   25638.459448  
            Эквайринг          14734.644751   20723.157814  
Массовый    Банковская карта   11912.631687   30911.203026  
            Депозит           101680.067703  109580.268403  
            Кредит            310724.231052  526476.177132  
            РКО                96940.659699   92964.322441  
            Эквайринг          51522.470650   76749.072807  
Стандартный Банковская карта    8930.506181   20573.908977  
            Депозит            78804.586529   77452.480043  
            Кредит            224709.381560  377857.442886  
            РКО                67450.919956   64473.542687  
            Эквайринг          37835.028014   55723.162104

In [7]:
pl_df.pivot_table('pl', ['segment', 'product'], 'period').style.background_gradient()

In [8]:
pl_df.pivot_table(['pl', 'client_id'], ['segment', 'product'], 'period', aggfunc={'pl': ['mean', 'sum'], 'client_id': 'count'}).style.background_gradient()

#### 2
Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.

In [9]:
pl_df.pivot_table(['pl', 'client_id'], 'segment', 'period', aggfunc={'pl': 'mean', 'client_id': 'count'}).style.background_gradient()

#### 3
Оценить отток клиентов с продуктов в периоде p_2

In [14]:
pt = pl_df.pivot_table('segment', ['product', 'client_id'], 'period', aggfunc='count', )

In [16]:
pt.head()

period                       p_1  p_2
product          client_id           
Банковская карта 00JURPZFUt  NaN  1.0
                 02O7Rlu7J8  1.0  NaN
                 02Uko8TXvH  1.0  NaN
                 03rEoXgXxH  1.0  NaN
                 05iexPlQKC  1.0  NaN

In [15]:
pt[pt.p_1.notna() & pt.p_2.isna()]\
.reset_index()\
.groupby('product')\
.agg('count')\
.iloc[:, :1]\
.rename(columns={'client_id': 'retention'})

period,retention
product,
Банковская карта,1079
Депозит,3824
Кредит,4084
РКО,1207
Эквайринг,2785


#### 4
Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании

In [18]:
df_merge = pl_df.merge(campaign)
df_merge.head()

,client_id,segment,period,product,pl,response,manager_id
0,XHgfHt3boG,Массовый,p_2,Кредит,35.244275,Согласие,CM_RCwv1r0OHO
1,sujQEPdgvp,Массовый,p_2,Кредит,121.858601,Не обработано менеджером,CM_lKxhD73Z6d
2,XRre8Aj7GK,Массовый,p_2,Кредит,43.220162,Недозвон,CM_c28lGRRaEI
3,faWYjccoeJ,Массовый,p_2,Кредит,6.691619,Согласие,CM_6ofody41cY
4,9k7VeLIbKU,Массовый,p_2,Кредит,22.162824,Согласие,CM_pMFgOhtUKD


In [25]:
df_merge.groupby('manager_id').pl.agg('sum').sort_values(ascending=False)

manager_id
CM_2755B4KTu4    22201.541290
CM_pMFgOhtUKD    20355.146751
CM_wjb2ArrpAw    19834.757303
CM_c28lGRRaEI    19565.680861
CM_gHP4ykbzTj    19539.450154
CM_R7BdTh0Svt    19518.652917
CM_6ofody41cY    19181.096090
CM_doxY9cvELI    19058.722925
CM_PzEEav4ic0    18478.955276
CM_lKxhD73Z6d    18366.204638
CM_eE0WMK1vVW    17996.289290
CM_onY6ha8qHv    17841.507248
CM_fQHTUUUkXc    17786.466100
CM_EiBeeRMhdA    17660.461325
CM_RCwv1r0OHO    17656.530108
CM_70HjN0EVqf    17602.269745
CM_zShYeVNFi0    17597.410380
CM_9vMKq9aqw9    16440.848277
CM_nJ7j5pAcMB    15828.418450
CM_dcTdLjqH2x    15266.406230
Name: pl, dtype: float64

In [12]:
pl_df.head(1)

,client_id,segment,period,product,pl
0,ZDosW1zUAi,Массовый,p_1,Кредит,9.018067


In [13]:
campaign.head(1)

,client_id,product,response,manager_id
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA
